# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 22 2022 8:30AM,240130,10,0085788720,ISDIN Corporation,Released
1,Jun 22 2022 8:30AM,240130,10,0085788735,ISDIN Corporation,Released
2,Jun 22 2022 8:30AM,240130,10,0085788739,ISDIN Corporation,Released
3,Jun 22 2022 8:30AM,240130,10,0085788740,ISDIN Corporation,Released
4,Jun 22 2022 8:30AM,240130,10,0085788746,ISDIN Corporation,Released
5,Jun 22 2022 8:30AM,240130,10,0085788753,ISDIN Corporation,Released
6,Jun 22 2022 8:30AM,240128,10,SONSB0001826,"Nextsource Biotechnology, LLC",Released
7,Jun 22 2022 8:30AM,240128,10,SONSB0001827,"Nextsource Biotechnology, LLC",Released
8,Jun 22 2022 8:30AM,240128,10,SONSB0001828,"Nextsource Biotechnology, LLC",Released
9,Jun 22 2022 8:30AM,240128,10,SOTAH0000710,"Nextsource Biotechnology, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,240126,Released,1
40,240127,Released,1
41,240128,Released,4
42,240129,Released,1
43,240130,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240126,NaN,NaN,1.0
240127,NaN,NaN,1.0
240128,NaN,NaN,4.0
240129,NaN,NaN,1.0
240130,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240030,0.0,0.0,1.0
240034,0.0,0.0,9.0
240037,14.0,4.0,4.0
240048,0.0,23.0,1.0
240061,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240030,0,0,1
240034,0,0,9
240037,14,4,4
240048,0,23,1
240061,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240030,0,0,1
1,240034,0,0,9
2,240037,14,4,4
3,240048,0,23,1
4,240061,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240030,,,1
1,240034,,,9
2,240037,14,4,4
3,240048,,23,1
4,240061,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 22 2022 8:30AM,240130,10,ISDIN Corporation
6,Jun 22 2022 8:30AM,240128,10,"Nextsource Biotechnology, LLC"
10,Jun 22 2022 8:30AM,240122,10,Emerginnova
14,Jun 22 2022 8:30AM,240117,10,Yusen – 3D Matrix
15,Jun 22 2022 8:30AM,240119,10,Yusen – 3D Matrix
16,Jun 21 2022 4:18PM,240129,12,HealthCaps LLC
17,Jun 21 2022 4:10PM,240127,19,"Propella Therapeutics, Inc."
18,Jun 21 2022 4:01PM,240126,19,"Propella Therapeutics, Inc."
19,Jun 21 2022 3:58PM,240125,16,Sartorius Stedim North America
20,Jun 21 2022 3:57PM,240124,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 22 2022 8:30AM,240130,10,ISDIN Corporation,,,6
1,Jun 22 2022 8:30AM,240128,10,"Nextsource Biotechnology, LLC",,,4
2,Jun 22 2022 8:30AM,240122,10,Emerginnova,,,4
3,Jun 22 2022 8:30AM,240117,10,Yusen – 3D Matrix,,,1
4,Jun 22 2022 8:30AM,240119,10,Yusen – 3D Matrix,,,1
5,Jun 21 2022 4:18PM,240129,12,HealthCaps LLC,,,1
6,Jun 21 2022 4:10PM,240127,19,"Propella Therapeutics, Inc.",,,1
7,Jun 21 2022 4:01PM,240126,19,"Propella Therapeutics, Inc.",,,1
8,Jun 21 2022 3:58PM,240125,16,Sartorius Stedim North America,,,1
9,Jun 21 2022 3:57PM,240124,19,"GCH Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 8:30AM,240130,10,ISDIN Corporation,6,,
1,Jun 22 2022 8:30AM,240128,10,"Nextsource Biotechnology, LLC",4,,
2,Jun 22 2022 8:30AM,240122,10,Emerginnova,4,,
3,Jun 22 2022 8:30AM,240117,10,Yusen – 3D Matrix,1,,
4,Jun 22 2022 8:30AM,240119,10,Yusen – 3D Matrix,1,,
5,Jun 21 2022 4:18PM,240129,12,HealthCaps LLC,1,,
6,Jun 21 2022 4:10PM,240127,19,"Propella Therapeutics, Inc.",1,,
7,Jun 21 2022 4:01PM,240126,19,"Propella Therapeutics, Inc.",1,,
8,Jun 21 2022 3:58PM,240125,16,Sartorius Stedim North America,1,,
9,Jun 21 2022 3:57PM,240124,19,"GCH Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 8:30AM,240130,10,ISDIN Corporation,6,,
1,Jun 22 2022 8:30AM,240128,10,"Nextsource Biotechnology, LLC",4,,
2,Jun 22 2022 8:30AM,240122,10,Emerginnova,4,,
3,Jun 22 2022 8:30AM,240117,10,Yusen – 3D Matrix,1,,
4,Jun 22 2022 8:30AM,240119,10,Yusen – 3D Matrix,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 8:30AM,240130,10,ISDIN Corporation,6,NaN,NaN
1,Jun 22 2022 8:30AM,240128,10,"Nextsource Biotechnology, LLC",4,NaN,NaN
2,Jun 22 2022 8:30AM,240122,10,Emerginnova,4,NaN,NaN
3,Jun 22 2022 8:30AM,240117,10,Yusen – 3D Matrix,1,NaN,NaN
4,Jun 22 2022 8:30AM,240119,10,Yusen – 3D Matrix,1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 8:30AM,240130,10,ISDIN Corporation,6,0.0,0.0
1,Jun 22 2022 8:30AM,240128,10,"Nextsource Biotechnology, LLC",4,0.0,0.0
2,Jun 22 2022 8:30AM,240122,10,Emerginnova,4,0.0,0.0
3,Jun 22 2022 8:30AM,240117,10,Yusen – 3D Matrix,1,0.0,0.0
4,Jun 22 2022 8:30AM,240119,10,Yusen – 3D Matrix,1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3601400,79,16.0,14.0
102,240089,1,0.0,0.0
12,240129,1,0.0,0.0
15,480114,2,23.0,0.0
16,720362,8,0.0,0.0
18,480215,2,0.0,0.0
19,1920864,23,7.0,0.0
20,720252,29,37.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3601400,79,16.0,14.0
1,102,240089,1,0.0,0.0
2,12,240129,1,0.0,0.0
3,15,480114,2,23.0,0.0
4,16,720362,8,0.0,0.0
5,18,480215,2,0.0,0.0
6,19,1920864,23,7.0,0.0
7,20,720252,29,37.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,79,16.0,14.0
1,102,1,0.0,0.0
2,12,1,0.0,0.0
3,15,2,23.0,0.0
4,16,8,0.0,0.0
5,18,2,0.0,0.0
6,19,23,7.0,0.0
7,20,29,37.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,79.0
1,102,Released,1.0
2,12,Released,1.0
3,15,Released,2.0
4,16,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20
Status,,,,,,,,
Completed,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,16.0,0.0,0.0,23.0,0.0,0.0,7.0,37.0
Released,79.0,1.0,1.0,2.0,8.0,2.0,23.0,29.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20
0,Completed,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,16.0,0.0,0.0,23.0,0.0,0.0,7.0,37.0
2,Released,79.0,1.0,1.0,2.0,8.0,2.0,23.0,29.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20
0,Completed,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,16.0,0.0,0.0,23.0,0.0,0.0,7.0,37.0
2,Released,79.0,1.0,1.0,2.0,8.0,2.0,23.0,29.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()